# Chinook Music Sales
For this project we will be querying the chinook database to answer certain hypothetical business questions. The questions we will be answering are as follow:

- In our first scenario,we will be trying to figure out the albums to add to our store from a number of artists whose music we don't have in our store. This decision will be guided by the genre that sells the most.
- We will try to evaluate the performance of our customer support agents using data from our database.
- Analyse sales data for each country.
- Compare album purchase to individual track purchases to figure out what sales strategy we can implement to drive revenue.
## Schema Diagram
<img src="https://s3.amazonaws.com/dq-content/191/chinook-schema.svg" alt="isolated" width="600"/>

In [14]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [15]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Sales by Genre in USA
Lets investigate which new albums to purchase based on what genre has the best sales numbers.

Require queries from:
- customer to get sales country = USA
- invoice to get total sales
- invoice_line to link invoice to track
- track to link invoice_line to genre
- genre to get genre name

In [16]:
%%sql
SELECT
    *
FROM GENRE;

Done.


genre_id,name
1,Rock
2,Jazz
3,Metal
4,Alternative & Punk
5,Rock And Roll
6,Blues
7,Latin
8,Reggae
9,Pop
10,Soundtrack


In [17]:
%%sql
SELECT
    t.track_id,
    g.name AS genre,
    COUNT(*) AS total
FROM track t
LEFT JOIN genre g ON g.genre_id = t.genre_id
GROUP BY genre
LIMIT 10;

Done.


track_id,genre,total
3478,Alternative,40
2818,Alternative & Punk,332
2590,Blues,81
660,Bossa Nova,15
3502,Classical,74
3429,Comedy,17
3364,Drama,64
1056,Easy Listening,24
3358,Electronica/Dance,30
1304,Heavy Metal,28


In [18]:
%%sql
SELECT
    il.invoice_id,
    il.track_id
FROM invoice_line il
LEFT JOIN track t ON t.track_id = il.track_id
LIMIT 10;

Done.


invoice_id,track_id
1,1158
1,1159
1,1160
1,1161
1,1162
1,1163
1,1164
1,1165
1,1166
1,1167


In [19]:
%%sql
SELECT
    il.invoice_line_id,
    il.track_id,
    i.billing_country,
    g.name,
    COUNT(g.name) as quanitity,
    COUNT(g.name)*100/(SELECT COUNT(*) 
    FROM invoice_line il
    LEFT JOIN invoice i ON i.invoice_id = il.invoice_id
    where i.billing_country = 'USA') as percentage
FROM invoice_line il
LEFT JOIN invoice i ON il.invoice_id = i.invoice_id
LEFT JOIN track t ON t.track_id = il.track_id
LEFT JOIN genre g ON g.genre_id = t.genre_id
WHERE i.billing_country = 'USA'
GROUP BY g.name
ORDER BY percentage DESC;

Done.


invoice_line_id,track_id,billing_country,name,quanitity,percentage
4742,2738,USA,Rock,561,53
4671,481,USA,Alternative & Punk,130,12
4743,1126,USA,Metal,124,11
4740,3465,USA,R&B/Soul,53,5
4635,3387,USA,Alternative,35,3
4617,202,USA,Blues,36,3
4663,1109,USA,Latin,22,2
4669,3263,USA,Pop,22,2
4667,1036,USA,Easy Listening,13,1
4453,3310,USA,Hip Hop/Rap,20,1


### Conclusion
Based on results above, rock and punk album should be sold.

## Analyzing Employee Sales Performance
The goal here is to find out which employee has performed the best and also to find out the underlying factor behind the employee's performance.

In [20]:
%%sql
SELECT 
    e.first_name || " " || e.last_name as employee,
    e.title as title,
    e.state as state,
    e.country as country,
    e.hire_date as hire_date,
    sum(i.total) as total
FROM invoice i
LEFT JOIN customer c ON c.customer_id = i.customer_id
LEFT JOIN employee e ON e.employee_id = c.support_rep_id
GROUP BY e.employee_id
LIMIT 100;
    

Done.


employee,title,state,country,hire_date,total
Jane Peacock,Sales Support Agent,AB,Canada,2017-04-01 00:00:00,1731.5100000000039
Margaret Park,Sales Support Agent,AB,Canada,2017-05-03 00:00:00,1584.0000000000034
Steve Johnson,Sales Support Agent,AB,Canada,2017-10-17 00:00:00,1393.920000000002


### Conclusion
While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), looking at the difference in location but the more sales total relates to length at company.

## Analyzing Sales by Country
We are going to find out the average sales value customers from the different countries as well as the average order value. Note: countries that did not have more than 1 customer were grouped together as Others.

In [21]:
%%sql
CREATE VIEW country_sales AS
SELECT 
    c.country as country,
    CASE
        WHEN COUNT(DISTINCT c.customer_id) = 1 THEN 'Other'
        ELSE c.country
    END AS sorted_country,
    COUNT(DISTINCT c.customer_id) AS total_customers,
    SUM(i.total)/COUNT(DISTINCT c.customer_id) AS avg_sales,
    SUM(i.total)/COUNT(i.invoice_id) AS avg_order,
    CASE
        WHEN COUNT(DISTINCT c.customer_id) = 1 THEN 1
        ELSE 0
    END AS sorting
FROM invoice i
LEFT JOIN customer c ON c.customer_id = i.customer_id
GROUP BY c.country;



(sqlite3.OperationalError) table country_sales already exists
[SQL: CREATE VIEW country_sales AS
SELECT 
    c.country as country,
    CASE
        WHEN COUNT(DISTINCT c.customer_id) = 1 THEN 'Other'
        ELSE c.country
    END AS sorted_country,
    COUNT(DISTINCT c.customer_id) AS total_customers,
    SUM(i.total)/COUNT(DISTINCT c.customer_id) AS avg_sales,
    SUM(i.total)/COUNT(i.invoice_id) AS avg_order,
    CASE
        WHEN COUNT(DISTINCT c.customer_id) = 1 THEN 1
        ELSE 0
    END AS sorting
FROM invoice i
LEFT JOIN customer c ON c.customer_id = i.customer_id
GROUP BY c.country;]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [22]:
%%sql
SELECT
    sorted_country as country,
    sum(total_customers) as total_customers,
    AVG(avg_sales) as avg_sales,
    AVG(avg_order) as avg_order
FROM country_sales
GROUP BY sorted_country
ORDER BY sorting, total_customers DESC;

Done.


country,total_customers,avg_sales,avg_order
USA,13,80.0376923076923,7.942671755725189
Canada,8,66.94875000000002,7.047236842105265
Brazil,5,85.53600000000002,7.0111475409836075
France,5,77.81399999999998,7.781399999999998
Germany,4,83.655,8.161463414634147
United Kingdom,3,81.84,8.768571428571429
Czech Republic,2,136.62000000000003,9.108000000000002
India,2,91.57499999999999,8.72142857142857
Portugal,2,92.56500000000001,6.383793103448276
Other,15,72.996,7.445071062271063


Based on the data, there may be opportunity in the following countries:

- Czech Republic
- United Kingdom
- India

It's worth keeping in mind that because the amount of data from each of these countries is relatively low. Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.

## Comparing Album Purchases to Tracks Purchase
The management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [23]:
%%sql
WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.